In [1]:
# this code is for draw the performance graph of ODE under Distributed model

In [3]:
np.random.seed(1)
A = 2
M = 6

def getPij(a):
    temp = np.random.rand(A,A)
    return (temp/sum(temp)).T
Pij = getPij(A)
ArrLst = np.random.rand(A)
Beta = 0.3
RhoMtx = np.random.rand(A, A)

Theta = 1.0
Mu =1.0
N = 1

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sp
from math import factorial
from tqdm import tqdm
import csv
from scipy.sparse import csr_matrix
from scipy.sparse import identity
from scipy.sparse.linalg import spsolve

class perfDisODE():
    def __init__(self, A, M, Pij, ArrLst, RhoMtx, Beta, Mu, N, Theta): 
        # init parameters
        self.A, self.M, self.Pij, self.ArrLst, self.RhoMtx, self.Beta, self.Mu, self.N, self.Theta = \
            A, M, Pij, ArrLst, RhoMtx, Beta, Mu, N, Theta
        # init state dict
        self.State = self.getStateDict()
        self.n_state = len(self.State)
    
    def getStateDict(self):
        State = {}
        index = [0]
        def temp(x, i, index, l):
            x.append(i)
            generate_R(x.copy(), index, l)
        def generate_R(s, index, l):
            if len(s)==self.A+self.A+self.A**2 and sum(s)==self.M:
                State[tuple(s+[l])] = index[0]
                index[0] += 1
            elif len(s)>self.A+3+self.A**2 or sum(s)>self.M:
                return 0
            else:
                for i in range(self.M+1):
                    temp(s.copy(), i, index, l)
        for l in range(self.A):
            for i in range(self.M+1):
                generate_R([i], index, l)
        return State
    
    def getMtx(self):
        State = self.State
        n_state = self.n_state
        A, M, Pij, ArrLst, RhoMtx, Beta, Mu, N, Theta = \
        self.A, self.M, self.Pij, self.ArrLst, self.RhoMtx, self.Beta, self.Mu, self.N, self.Theta
        def INi(ni):
            if ni > 0: return 1
            else: return 0
        def IL(s):
            if s[A+s[-1]] == 0: return 1
            else: return 0
        def IS(s):
            # 判断是不是都是在0到M之间
            if tuple(s) in State.keys(): return 1
            else: return 0

        def arrRateIn(s):
            n, col, val = 0, [], []
            for i in range(A):
                for j in range(A):
                    tempS = list(s)
                    # Ni, Rij
                    y = 2*A + i*A + j
                    a1 = tempS[i] = tempS[i] + 1
                    a2 = tempS[y] = tempS[y] - 1
                    if IS(tempS):
                        n += 1
                        col.append(State[tuple(tempS)])
                        val.append(ArrLst[i]*Pij[i][j])
            #         print(tempS)
            # print(n, col, val)
            # print('arr-----------')
            return n, col, val

        def backRateIn(s):
            n, col, val = 0, [], []
            for i in range(A):
                for j in range(A):
                    tempS = list(s)
                    # Ni, Rij
                    y = 2*A + j*A + i
                    a1 = tempS[i] = tempS[i] - 1
                    a2 = tempS[y] = tempS[y] + 1
                    if IS(tempS):
                        n += 1
                        col.append(State[tuple(tempS)])
                        val.append(RhoMtx[j][i]*a2*(1-Beta))
            #         print(tempS)
            # print(n, col, val)
            # print('back-----------')
            return n, col, val

        def broRateIn(s):
            n, col, val = 0, [], []
            for i in range(A):
                for j in range(A):
                    tempS = list(s)
                    #Rij, BP
                    y = 2*A + j*A + i
                    a1 = tempS[y] = tempS[y] + 1
                    a2 = tempS[A+i] = tempS[A+i] - 1
                    if IS(tempS):
                        n += 1
                        col.append(State[tuple(tempS)])
                        val.append(RhoMtx[j][i]*a1*Beta)
            #         print(tempS)
            # print(n, col, val)
            # print('bro-----------')
            return n, col, val

        def fixRateIn(s):
            n, col, val = 0, [], []
            tempS = list(s)
            # RC, DP
            L = s[-1]
            a1 = tempS[A+L] = tempS[A+L] + 1
            a2 = tempS[L] = tempS[L] - 1
            if IS(tempS):
                n += 1
                col.append(State[tuple(tempS)])
                val.append(Mu*min(tempS[A+L], N))
            # print(tempS)
            # print(n, col, val)
            # print('fix-----------')
            return n, col, val

        def movRateIn(s):
            n, col, val = 0, [], []
            tempS = list(s)
            index = (s[-1]-1) % A
            tempS[A+index] = 0
            tempS[-1] = index
            if sum(tempS[:-1]) == M:
                n += 1
                col.append(State[tuple(tempS)])
                val.append(Theta)
            # print(tempS)
            # print(n, col, val)
            # print('red-----------')
            return n, col, val

        def getRateOut(s):
            outRate = 0
            for i in range(A):
                if INi(s[i]):
                    outRate += ArrLst[i]
                else: continue
            for i in range(A):
                for j in range(A):
                    outRate += RhoMtx[i][j] * s[2*A+i*A+j]
            outRate += Mu*min(s[A+s[-1]], N) + Theta*IL(s)
            # inRate += sum(list(map(lambda a: a[0]*a[1], zip(x,y))))
            return -outRate

        def getRateIn(s):
            n, col, val = 0, [], []
            '''
            # customer arrival: arrRateIn
            # ride back: backRateIn
            # ride break down: broRateIn
            # gathering: gathRateIn
            # fixing: fixRateIn
            # redistributing: redRateIn
            '''
            for f in [arrRateIn, backRateIn, broRateIn, fixRateIn, movRateIn]:
                tempN, tempCol, tempVal = f(s)
                n += tempN
                col += tempCol
                val += tempVal

            return n, col, val

        def generateR():
            #R = csr_matrix((S,S), dtype=np.float)
            Row, Col, Value = [], [], []
            for k, s in enumerate(State):
                '''
                number of row: n
                row number: k
                column number: col
                value: data
                '''
                # 加1
                if k==n_state-1: # collect the last row as a test instance
                    tempN, tempCol, tempVal = getRateIn(s)
                    tempCol += [k]
                    tempVal += [getRateOut(s)]
                else:            # generate the mtx
                    # set rate out for state s
                    Row += [k]
                    Col += [k]
                    Value += [getRateOut(s)]

                    # set rate in for state s
                    tempN, tempCol, tempVal = getRateIn(s)
                    Row += [k] * tempN
                    Col += tempCol
                    Value += tempVal

            Row += [k] * n_state
            Col += list(range(n_state))
            Value += [1] * n_state
            R = csr_matrix((Value, (Row, Col)), dtype=np.float) #.toarray()
            testArr = csr_matrix((tempVal, ([0]*(tempN+1), tempCol)), dtype=np.float)
            return R, testArr
        return generateR()
    
    def getPortionState(self):
        BalanceMtx, testArr = self.getMtx()
        b = np.array([0]*(self.n_state-1) + [1])
        x = spsolve(BalanceMtx, b)
        #assert(BalanceMtx.toarray()[-1].dot(x) == 1.0 and testArr.toarray().dot(x) == 0)
        portionState = {}
        for k,s in enumerate(self.State):
            portionState[s] = x[k]
        return portionState
    
    def getPerf(self, para):
        def getLost(state):
            _sum = 0.0
            for n,s in enumerate(range(self.A)):
                if state[n] == 0: 
                    _sum += self.ArrLst[n]
            return _sum / sum(self.ArrLst)
        def getResult(n,b,i,l):
            re = []
            for _ in [n,b,i,l]:
                re.append([self.para, _, 0])
            return re
        self.para = para
        portionState = self.getPortionState()
        normalBikes, brokenBikes, idle, lost = 0,0,0,0
        for k,s in enumerate(portionState):
            por = portionState[s]
            normalBikes += sum(s[:self.A]+s[2*self.A:-1]) * por
            brokenBikes += sum(s[self.A:2*self.A]) * por
            if s[self.A+s[-1]] == 0: idle += por
            lost += getLost(s) * por
        return getResult(normalBikes, brokenBikes, idle, lost)
    
instance = perfDisODE(A, M, Pij, ArrLst, RhoMtx, Beta, Mu, N, Theta)
instance.getPerf(Beta)

[[0.3, 5.931924899668865, 0],
 [0.3, 0.06807510033113703, 0],
 [0.3, 0.9559669569007838, 0],
 [0.3, 0.3861137785296975, 0]]